In [1]:
# https://github.com/openai/openai-cookbook/blob/main/examples/How_to_format_inputs_to_ChatGPT_models.ipynb
# https://stackoverflow.com/questions/77285102/how-to-format-a-few-shot-prompt-for-gpt4-chat-completion-api

In [2]:
# pip install --upgrade openai
# OLLAMA_MODELS=/usr/share/d_ollama/.ollama/models/hf_model ollama serve

In [3]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)


In [4]:
import warnings


import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import config.config as conf

import json
from itertools import chain
warnings.filterwarnings("ignore", category=DeprecationWarning)
# https://wikidocs.net/233348

In [5]:
# # Example OpenAI Python library request
# MODEL = "gpt-4o-2024-08-06"
# response = client.chat.completions.create(
#     model=MODEL,
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": "Explain asynchronous programming in the style of the pirate Blackbeard."},
#     ],
#     temperature=0,
# )

In [6]:
# print(json.dumps(json.loads(response.model_dump_json()), indent=4))

In [7]:
df = pd.read_csv('../../data/q_output_code_y.csv', index_col = 0)

In [8]:
df.shape

(76, 3)

In [9]:
df['question'][0]

'<Title>Why joblib is not recommended when saving keras model?</Title>. <Question>According to this keras documentation, pickle is not recommended to save keras mode, and since joblib.dump() and joblib.load() are based on the Python pickle serialization model, joblib is also not recommended to save keras model. What is the reason for that ?\n</Question>'

In [10]:
# sys_promt="""\nYou are an expert in analyzing and categorizing the difficulty level of Python-related programming questions based on examples below. 
# Questions are sourced from the "StackOverflow" community using the keyword "Python."\n"""

In [11]:
# user_prompt_front = """\nFor the given (target) post that is marked by
#     <target_post> </target_post>, please let me know the
#     "Difficulty Level" of the target post, where the
#     "Difficulty Level" can be one of the followings: (0) Basic, (1)
#     Intermediate, and (2) Advanced\n"""

In [12]:
# user_prompt_back = """\nplease output only the difficulty label of the target post, where it can only be one of, (0) Basic, (1)
#     Intermediate, and (2) Advanced, 
#     and mark '0' if it is basic, '1' if is is intermediate, and '2' if it is advanced question. 
#     Please refer to the samples below that may be helpful to measure the baseline of the difficulty labels, where each
#     of the sample is delimited by "----"\n
#     """

In [13]:
diff_dict = {'Difficulty Level : Basic':0 ,
 'Difficulty Level : Intermediate':1, 
 'Difficulty Level : Advanced':2}

In [14]:
df['answer_encode'] = df['answer'].apply(lambda x : diff_dict[x])

In [15]:
df.head()

,id,question,answer,answer_encode
0,74693871,<Title>Why joblib is not recommended when savi...,Difficulty Level : Advanced,2
1,74700764,<Title>replace multiple strings in a list with...,Difficulty Level : Basic,0
2,74727945,<Title>pandas unit test AssertionError: DataFr...,Difficulty Level : Intermediate,1
3,74751254,<Title>Removing all duplicate images with diff...,Difficulty Level : Intermediate,1
4,74760874,<Title>How do I extract a binary pattern from ...,Difficulty Level : Intermediate,1


In [16]:
diff_idx = {x : list(df[df['answer']==x].index) for x in list(diff_dict.keys())}

In [17]:
diff_s_idx = {}
for key, value in diff_idx.items():
    print(key, value)
    dic_col = f'{key}_sample_idx'
    diff_population = value
    diff_s_idx[dic_col] = np.random.choice(diff_population, size=3, replace=False)

Difficulty Level : Basic [1, 7, 11, 16, 17, 18, 21, 22, 29, 31, 32, 33, 39, 41, 43, 44, 54, 55, 56, 57, 60, 65, 67, 69, 70, 72, 73]
Difficulty Level : Intermediate [2, 3, 4, 5, 6, 8, 9, 10, 12, 14, 19, 20, 23, 24, 25, 26, 27, 28, 30, 34, 35, 36, 38, 40, 42, 45, 46, 47, 48, 50, 51, 52, 53, 58, 59, 61, 63, 64, 68, 71]
Difficulty Level : Advanced [0, 13, 15, 37, 49, 62, 66, 74, 75]


In [18]:
diff_s_idx

{'Difficulty Level : Basic_sample_idx': array([43, 22, 11]),
 'Difficulty Level : Intermediate_sample_idx': array([27, 42, 50]),
 'Difficulty Level : Advanced_sample_idx': array([13, 37, 75])}

In [19]:
fewshot_q_id = list(chain.from_iterable(diff_s_idx.values()))


In [20]:
eval_q_id =np.setdiff1d(list(df.index), fewshot_q_id)

In [21]:
examples = []
for idx in fewshot_q_id:
# for idx in [19]:
    temp_dict = {"question" : str(df.loc[idx, 'question']),
                 "answer"   : str(df.loc[idx, 'answer_encode'])}
    examples.append(temp_dict)

In [22]:
examples

[{'question': "<Title>How do I remove time duration as outliers from a column in a dataframe?</Title>. <Question>I want to remove outliers from a datetime columns after applying difference between two datetime to get the duration. I applied boxplot to get the outliers, but, I cannot remove the outliers from the new duration column. here is the code.\n```df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])\ndf['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])\n\ndf['trip duration'] = df['tpep_dropoff_datetime'] -df['tpep_pickup_datetime']\n# convert time duration to integer.\ndf['trip duration'] = int(df['trip duration'].dp.strftime('%Y%m%d%H%M%S')) \nfor the distance column\nfig = plt.figure(figsize =(10, 7))\nplt.boxplot(df['trip_distance'])\nplt.show()\n# for the duration column\nfig = plt.figure(figsize =(10, 7))\nplt.boxplot(df['trip duration'])\nplt.show()\n\n\n# drop the outliers\n\ndf_2= df[df['trip_distance'] <; 6] \n\ndf_2= df[df['trip d

In [23]:
sys_promt = """
You are an expert in analyzing and categorizing the "Difficulty Level" of Python-related questions.
please let me know the
    "Difficulty Level" of the target post, where the
    "Difficulty Level" can be one of the followings: (1)
    Advanced, which is for a difficult one, (2) Intermediate,
    which is for a somewhat difficult one, and (3) Basic,
    which is an easy one.

***Instructions***

***1. Analyze the examples of questions***
- refer to the samples below that may be helpful to measure the baseline of the "Difficulty Level", where each
of the sample is delimited by "----"

***2.Measure the "Difficulty Level" of target question***
- For the given (target) post that is marked by <target_post> </target_post>
- please let me know the "Difficulty Level" of the target post, where the "Difficulty Level" can be one of the followings : (1)
    Advanced, which is for a difficult one, (2) Intermediate,
    which is for a somewhat difficult one, and (3) Basic,
    which is an easy one.

***3.Print out the "Difficulty Level"***
- no explanation is needed for "Difficulty Level"
- Expected  output
    if question == Easy or Basic or Bigginer level:
        Difficulty Level: 0
    elif question == intermediate level:
        Difficulty Level: 1
    elif question == advanced level:
        Difficulty Level: 2
"""

In [24]:
q_prompt = """\nHere is the examples of question"""
for q_a in examples : 
    q_prompt = q_prompt+"""\n----\n"""
    q_prompt = q_prompt+f"Difficulty Level : {q_a['answer']}\n"
    q_prompt = q_prompt+"""<post>\n"""
    q_prompt = q_prompt+f"{q_a['question']}\n"
    q_prompt = q_prompt+"""</post>\n"""
q_prompt = q_prompt+"""----\n"""

In [25]:
eval_q_idx= np.random.choice(eval_q_id, size=1, replace=False)

In [26]:
target_post="""\nHere is the target post. Answer the "Difficulty Level".\n"""
target_post = target_post+"""\n<target_post>\n"""
target_post = target_post+df.loc[eval_q_idx, 'question'].values[0]+'\n'
target_post = target_post+"""</target_post>\n"""


In [27]:
# tot_prompt = sys_promt+user_prompt_front+target_post+user_prompt_back+q_prompt
tot_prompt = q_prompt+target_post

In [28]:
print(tot_prompt)


Here is the examples of question
----
Difficulty Level : 0
<post>
<Title>How do I remove time duration as outliers from a column in a dataframe?</Title>. <Question>I want to remove outliers from a datetime columns after applying difference between two datetime to get the duration. I applied boxplot to get the outliers, but, I cannot remove the outliers from the new duration column. here is the code.
```df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

df['trip duration'] = df['tpep_dropoff_datetime'] -df['tpep_pickup_datetime']
# convert time duration to integer.
df['trip duration'] = int(df['trip duration'].dp.strftime('%Y%m%d%H%M%S')) 
for the distance column
fig = plt.figure(figsize =(10, 7))
plt.boxplot(df['trip_distance'])
plt.show()
# for the duration column
fig = plt.figure(figsize =(10, 7))
plt.boxplot(df['trip duration'])
plt.show()


# drop the outliers

df_2= df[df['trip_distanc

In [29]:
with open("output.json", "w") as file:
    json.dump(tot_prompt, file)

In [30]:
df.loc[eval_q_idx, :]

,id,question,answer,answer_encode
69,77404924,<Title>How would I make the turtle player move...,Difficulty Level : Basic,0


In [31]:
from ollama import chat
# from pathlib import Path

response = chat(
  model='llama-3.1-70b-instruct-lorablated.Q4_K_M:latest',
  messages=[
    # {"role": "system", "content": sys_promt},
    {'role': 'user','content': tot_prompt}
    
  ],
)


In [32]:
print(response['message']['content'])

Difficulty Level: 1



In [ ]:
sys_promt + 